# SparkContext - number of workers and lazy evaluation

## Checking the impact of number of workers
While initializing the `SparkContext`, we can specify number of worker nodes. Generally, it is recommended to have one worker per core of the machine. But it can be smaller or larger. In the following code, we will examine the impact of number of worker cores on some parallelized operation.

In [1]:
import findspark
findspark.init()
from time import time
from pyspark import SparkContext

In [18]:
for j in range(1,10):
    sc= SparkContext(master = "local[%d]"%(j))
    t0=time()
    for i in range(10):
        sc.parallelize([1,2]*10000).reduce(lambda x,y:x+y)
    print(f"{j} executors, time = {time()-t0}")
    sc.stop()

1 executors, time = 2.7400147914886475
2 executors, time = 2.6245410442352295
3 executors, time = 2.8254308700561523
4 executors, time = 2.8230197429656982
5 executors, time = 3.0630319118499756
6 executors, time = 4.093182325363159
7 executors, time = 3.34519624710083
8 executors, time = 3.6763877868652344
9 executors, time = 4.376622200012207


#### We observe that it takes almost double time for 1 worker, and after that time reduces to a flat level for 2,3,4 workers etc. This is because this code run on a Linux virtual box using only 2 cores from the host machine. If you run this code on a machine with 4 cores, you will see benefit upto 4 cores and then the flattening out of the time taken. It also become clear that using more than one worker per core is not beneficial as it just does context-switching in that case and does not speed up the parallel computation.

## Showing the essence of _lazy_ evaluation
![](https://qph.fs.quoracdn.net/main-qimg-d49dcf35ecb7eecfc6e5b39493a0e086-c)

In [3]:
sc = SparkContext(master="local[2]")

### Make a RDD with 1 million elements

In [4]:
%%time
rdd1 = sc.parallelize(range(1000000))

CPU times: user 1.72 ms, sys: 760 µs, total: 2.48 ms
Wall time: 3.66 ms


### Some computing function - `taketime`

In [5]:
from math import cos
def taketime(x):
    [cos(j) for j in range(100)]
    return cos(x)

### Check how much time is taken by `taketime` function

In [6]:
%%time
taketime(2)

CPU times: user 395 µs, sys: 173 µs, total: 568 µs
Wall time: 42.3 ms


-0.4161468365471424

### Now do the `map` operation on the function

In [7]:
%%time
interim = rdd1.map(lambda x: taketime(x))

CPU times: user 83 µs, sys: 36 µs, total: 119 µs
Wall time: 124 µs


#### How come each taketime function takes 45.8 us but the map operation with a 10000 element RDD also took similar time?<br><br>Because of _lazy_ evaluation i.e. nothing was computed in the previous step, just a plan of execution was made. The variable `interim` does not point to a data structure, instead it points to a plan of execution, expressed as a dependency graph. The dependency graph defines how RDDs are computed from each other.

### Let's see the "Dependency Graph" using `toDebugString` method

In [8]:
print(interim.toDebugString().decode())

(2) PythonRDD[1] at RDD at PythonRDD.scala:53 []
 |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289 []


![](https://raw.githubusercontent.com/tirthajyoti/Spark-with-Python/master/Images/RDD_dependency_graph.PNG)

### The actual execution by `reduce` method

In [9]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

output = -0.2887054679684353
CPU times: user 6.98 ms, sys: 6.4 ms, total: 13.4 ms
Wall time: 8.96 s


In [10]:
1000000*31e-6

31.0

#### It is less than what we would have expected considering 1 million operations with the `taketime` function. This is the result of parallel operation of 2 cores.

### Now, we have not saved (materialized) any intermediate results in `interim`, so another simple operation (e.g. counting elements > 0) will take almost same time

In [11]:
%%time
print(interim.filter(lambda x:x>0).count())

500000
CPU times: user 8.32 ms, sys: 3.69 ms, total: 12 ms
Wall time: 7.96 s


## Caching to reduce computation time on similar operation (spending memory)

### Run the same computation as before with `cache` method to tell the dependency graph to plan for caching

In [12]:
%%time
interim = rdd1.map(lambda x: taketime(x)).cache()

CPU times: user 7.62 ms, sys: 3.21 ms, total: 10.8 ms
Wall time: 1.15 s


In [13]:
print(interim.toDebugString().decode())

(2) PythonRDD[4] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289 [Memory Serialized 1x Replicated]


In [14]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

output = -0.2887054679684353
CPU times: user 7.44 ms, sys: 3.24 ms, total: 10.7 ms
Wall time: 8.9 s


### Now run the same `filter` method with the help of cached result

In [15]:
%%time
print(interim.filter(lambda x:x>0).count())

500000
CPU times: user 14.9 ms, sys: 6.28 ms, total: 21.2 ms
Wall time: 833 ms


#### This time it took much shorter time due to cached result, which it could use to compare to 0 and count easily.

In [16]:
sc.stop()